In [1]:
include("./dataManager.jl")

# Implementation of the Bottom Up Network with two Subversions: 
# BK with different Kernelsize and BF with different feature size
using Flux, Statistics
using Flux: onehot, onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON
import LinearAlgebra: norm
using NNlib

using MAT # needs installation of Pkg.add("MAT")
using PyPlot # pip install Matplotlib Pkg.add("PyPlot")
using ..dataManager: make_batch
# using FeedbackConvNets
using Base
# using CuArrays

norm(x::TrackedArray{T}) where T = sqrt(sum(abs2.(x)) + eps(T)) 

norm (generic function with 16 methods)

In [2]:
# PARAMETERS
# ----------
const batch_size = 100
const n = 5
const c = 1
const alpha = Float32(10^-4)
const beta = 0.5f0
const lambda = 0.0005f0

const kernelsize = (3, 3)
const featuresize = 32

const momentum = 0.9f0
learningRate = 0.01f0
const epochs = 100
const decay_rate = 0.1f0
const decay_step = 40

train_idx = 0

0

In [3]:
@info("Constructing model...")
model = Chain(
    # HIDDEN LAYER 1
    # Input Image 32x32x1xN
    Conv(kernelsize, 1=>featuresize, pad=(1,1), relu),
    # local response normalization
    x -> localResponseNorm(x),
    MaxPool((2, 2), stride=(2, 2)),
    
    # HIDDEN LAYER 2
    # Input Image 16x16x32xN
    Conv(kernelsize, featuresize=>featuresize, pad=(1,1), relu),
    # local response normalization
    x -> localResponseNorm(x),
    MaxPool((16, 16), stride=(1, 1)),
    # reshape to 32xN
    x -> reshape(x, :, size(x, 4)),
    Dense(32, 10, σ),
)

# test the model (precomile it??)
model = model |> gpu

┌ Info: Constructing model...
└ @ Main In[3]:1


Chain(Conv((3, 3), 1=>32, NNlib.relu), getfield(Main, Symbol("##3#6"))(), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, NNlib.relu), getfield(Main, Symbol("##4#7"))(), MaxPool((16, 16), pad = (0, 0, 0, 0), stride = (1, 1)), getfield(Main, Symbol("##5#8"))(), Dense(32, 10, NNlib.σ))

In [4]:
# Input is either 32x32x32xN or 16x16x32xN
function localResponseNorm(x)
    w = zeros(Float32, size(x))
    for k = 1:size(x,3)
        # calculate the boundaries of the sum
        lower = Int32(max(1, floor(k-n/2)))
        upper = Int32(min(size(x,3), floor(k+n/2)))
        
        # square and sum 
        # w = map((x) -> x^2, x[:, :, lower:upper, :])
        norm = Tracker.data(x[:, :, lower:upper, :])
        norm = norm .^ 2
        # reduce to one 32x32x1xN or 16x16x1xN matrix
        norm = sum(norm, dims=3)
        norm = norm .* alpha
        norm = norm .+ c
        norm = norm .^ (-beta)
        w[:, :, k, :] = norm
    end   
    # a tracked array multiplied with a non tracked array returns a tracked array  
    x = w .* x
    return x
end

function loss(x, y)
    y_hat = model(x)
    return crossentropy(y_hat, y) + lambda * sum(norm, params(model))
end

# gets called after every batch (learning step)
function train_cb()
    global train_idx
    train_idx = train_idx + 1
    @info("now at batch $(train_idx)")
end

train_cb (generic function with 1 method)

In [5]:
# get the dataset
train_set, mean_img, std_img = make_batch(batch_size=10)
train_set = gpu.(train_set)
model(train_set[1][1])

@info("Training...")
optimizer = Momentum(learningRate, momentum)

# training loop
loss_val = loss(train_set[1][1], train_set[1][2])

Flux.train!(loss, params(model), train_set, optimizer, cb = train_cb)


┌ Info: Reading .mat file form source ../digitclutter/src/light_debris/light_debris_with_debris.mat
└ @ Main.dataManager C:\Users\Sebastian Vendt\Git\NNFeedbackOperations\Src\dataManager.jl:39
┌ Info: calculate mean and standart deviation of dataset
└ @ Main.dataManager C:\Users\Sebastian Vendt\Git\NNFeedbackOperations\Src\dataManager.jl:55
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
┌ Info: creating batches
└ @ Main.dataManager C:\Users\Sebastian Vendt\Git\NNFeedbackOperations\Src\dataManager.jl:73
┌ Info: Training...
└ @ Main In[5]:6
┌ Info: now at batch 1
└ @ Main In[4]:33
┌ Info: now at batch 2
└ @ Main In[4]:33
┌ Info: now at batch 3
└ @ Main In[4]:33
┌ Info: now at batch 4
└ @ Main In[4]:33
┌ Info: now at batch 5
└ @ Main In[4]:33
